In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommendation System').getOrCreate()

In [10]:
path = '/Users/wangyinuo/Desktop/DSCI551/project/yelp_dataset/yelp_academic_dataset_business.json'
# path = '/Users/wangyinuo/Desktop/DSCI551/project/test.csv'
df = spark.read.json(path)
# df = spark.read.load(path, header = True, format = "csv")
# df = spark.read.json(path, multiLine=True)

In [11]:
df.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [12]:
df.show()

+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|           city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|     10913 Bailey Rd|[,,,,,,, True,, T...|f9NumwFMBDn751xgF...|Active Life, Gun/...|      Cornelius|[11:0-20:0, 10:0-...|      1|   35.4627242|    -80.8526119|The Range At Lake...|      28031|          36|  3.5|   NC|
|8880 E Via Linda,...|[,,,,,,,,,,, True...|Yzvjg0SayhoZgCljU...|Health & Medical,...|     Scottsdale|                null|  

In [13]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col
temp = df
temp = temp.where(col("categories").isNotNull()).where(col("business_id").isNotNull())
temp = temp.rdd.map(lambda row : (row['business_id'], row["categories"].split(', '))).toDF(["business_id", "categories"])
temp.show()

+--------------------+--------------------+
|         business_id|          categories|
+--------------------+--------------------+
|f9NumwFMBDn751xgF...|[Active Life, Gun...|
|Yzvjg0SayhoZgCljU...|[Health & Medical...|
|XNoUzKckATkOD1hP6...|[Pets, Pet Servic...|
|6OAZjbxqM5ol29BuH...|[Hardware Stores,...|
|51M2Kk903DFYI6gnB...|[Home Services, P...|
|cKyLV5oWZJ2NudWgq...|[Auto Repair, Aut...|
|oiAlXZPIFm2nBCt0D...|[Dry Cleaning & L...|
|ScYkbYNkDgCneBrD9...|[Auto Repair, Oil...|
|pQeaRpvuhoEqudo3u...|[Ethnic Food, Foo...|
|EosRKXIGeSWFYWwpk...|[Martial Arts, Gy...|
|MbZMmwo-eL0Jnm_Yb...|[Contractors, Lan...|
|7Dv4_HAxsxvadEsT5...|[Automotive, Auto...|
|M_guz7Dj7hX0evS67...|[Desserts, Food, ...|
|JjJs3o60uQCfctDjs...|[Contractors, Hom...|
|kOICO53wbOiOJcKuC...|[Beauty & Spas, T...|
|rqcOZePlVvJP9Etzl...|[Local Services, ...|
|uZuh51lXu7tsrC8RA...|[Shopping, Shoe S...|
|nIEhsGbw0vJuYl05b...|[Event Planning &...|
|edQoeeBFUTYGwnUSE...|[Auto Customizati...|
|Vwo64kNYDjKi98gUU...|[Weight Lo

In [14]:
import pyspark.sql.functions as f
ls = temp.select(f.collect_list(temp.categories).alias('r')).collect()
ss = set()
for e in ls[0][0]:
    for ee in e:
        ss.add(ee)
print(ss)

{'Butcher', 'Dog Parks', 'Home Organization', 'Truck Rental', 'Trailer Dealers', 'South African', 'Machine Shops', 'Lawyers', 'Fabric Stores', 'Cryotherapy', 'Erotic Massage', 'Bakeries', 'Hearing Aid Providers', 'Taiwanese', 'Pet Transportation', 'Roof Inspectors', 'Soul Food', 'Diagnostic Services', 'Limos', 'Performing Arts', 'Gluten-Free', 'Country Dance Halls', 'Watch Repair', 'Orthotics', 'Veterans Organizations', 'Carpet Cleaning', 'Milkshake Bars', 'Pasta Shops', 'Advertising', 'DIY Auto Shop', 'Painters', 'Racing Experience', 'Experiences', 'Insurance', 'Music Production Services', 'Grilling Equipment', 'Trainers', 'Sports Betting', 'Water Stores', 'Calligraphy', 'Pawn Shops', 'Hostels', 'Couriers & Delivery Services', 'International', 'Osteopaths', 'Aircraft Repairs', 'Delicatessen', 'Electronics', 'Tenant and Eviction Law', 'Legal Services', 'Smog Check Stations', 'Check Cashing/Pay-day Loans', 'Auto Parts & Supplies', 'Tableware', 'Self-defense Classes', 'Organic Stores', '

In [15]:
yelp_category_json_path = '/Users/wangyinuo/Desktop/DSCI551/project/categories.json'
json_df = spark.read.json(yelp_category_json_path, multiLine=True)
json_df = json_df.select(json_df['parents'], json_df['title'])
json_df = json_df.rdd.map(lambda row : (','.join(row['parents']), row['title'])).toDF(["parents", 'title'])
json_df.show()
json_df = json_df.filter("parents LIKE '%restaurants%' or parents LIKE '%food%'")
json_df.show()

+-------------+-------------------+
|      parents|              title|
+-------------+-------------------+
|localservices|        3D Printing|
|      italian|          Abruzzese|
|         bars|      Absinthe Bars|
|         food|         Acai Bowls|
|      fashion|        Accessories|
| professional|        Accountants|
|    beautysvc|     Acne Treatment|
|             |        Active Life|
|       health|        Acupuncture|
|   physicians| Addiction Medicine|
|localservices|  Adoption Services|
|     shopping|              Adult|
|    education|    Adult Education|
|    nightlife|Adult Entertainment|
| professional|        Advertising|
|      fitness|     Aerial Fitness|
|        tours|       Aerial Tours|
|   medicalspa|      Aestheticians|
|  restaurants|             Afghan|
|  restaurants|            African|
+-------------+-------------------+
only showing top 20 rows

+-----------+--------------------+
|    parents|               title|
+-----------+--------------------+
|    

In [16]:
json_df = json_df.groupBy('parents').agg(f.collect_list('title').alias('titles'))
json_df.show()

+----------------+--------------------+
|         parents|              titles|
+----------------+--------------------+
|            food|[Acai Bowls, Back...|
|     restaurants|[Afghan, African,...|
|   food,shopping|             [Kiosk]|
|food,restaurants|      [Parent Cafes]|
|       arts,food|          [Wineries]|
+----------------+--------------------+



In [17]:
category_ls = json_df.select(f.collect_list(json_df.titles).alias('r')).collect()

In [18]:
category_ss = set()
for e in category_ls[0][0]:
    for ee in e:
        category_ss.add(ee)
print(category_ss)

{'Butcher', 'Dumplings', 'Fondue', 'Laotian', 'Flatbread', 'Waffles', 'Cambodian', 'Pan Asian', 'Indian', 'German', 'Laos', 'Traditional Swedish', 'Fishmonger', 'Signature Cuisine', 'Andalusian', 'Shaved Snow', 'Basque', 'Polynesian', 'Juice Bars & Smoothies', 'Persian/Iranian', 'Canadian (New)', 'Sugar Shacks', 'Indonesian', 'Cheesesteaks', 'Donairs', 'Lyonnais', 'Brazilian', 'Swiss Food', 'Catalan', 'Bakeries', 'Kombucha', 'Specialty Food', 'Taiwanese', 'Coffee & Tea', 'Soul Food', 'Black Sea', 'Eastern European', 'Hungarian', 'Buffets', 'Cideries', 'Delis', 'Gluten-Free', 'American (New)', 'Zapiekanka', 'Russian', 'Comfort Food', 'African', 'Hawaiian', 'International Grocery', 'Danish', 'Somali', 'Sri Lankan', 'Fischbroetchen', 'Milkshake Bars', 'Syrian', 'Sushi Bars', 'Ice Cream & Frozen Yogurt', 'Supper Clubs', 'Czech/Slovakian', 'Backshop', 'Oriental', 'Kopitiam', 'Yugoslav', 'Vegan', 'Bavarian', 'Food Court', 'Piadina', 'Night Food', 'Cafes', 'Milk Bars', 'Food Delivery Services

In [19]:
final_category_set = ss.intersection(category_ss)
print(final_category_set)

{'Butcher', 'Dumplings', 'Fondue', 'Laotian', 'Cambodian', 'Pan Asian', 'Indian', 'German', 'Signature Cuisine', 'Basque', 'Polynesian', 'Shaved Snow', 'Juice Bars & Smoothies', 'Persian/Iranian', 'Canadian (New)', 'Sugar Shacks', 'Indonesian', 'Cheesesteaks', 'Donairs', 'Brazilian', 'Swiss Food', 'Catalan', 'Bakeries', 'Kombucha', 'Specialty Food', 'Taiwanese', 'Coffee & Tea', 'Soul Food', 'Eastern European', 'Hungarian', 'Buffets', 'Cideries', 'Delis', 'Gluten-Free', 'American (New)', 'Russian', 'Comfort Food', 'African', 'Hawaiian', 'International Grocery', 'Sri Lankan', 'Milkshake Bars', 'Syrian', 'Sushi Bars', 'Supper Clubs', 'Ice Cream & Frozen Yogurt', 'Czech/Slovakian', 'Backshop', 'Vegan', 'Food Court', 'Thai', 'Cafes', 'Food Delivery Services', 'Nicaraguan', 'Pretzels', 'Bubble Tea', 'Kebab', 'Modern European', 'Scottish', 'Tapas/Small Plates', 'Acai Bowls', 'Coffee & Tea Supplies', 'Fish & Chips', 'Middle Eastern', 'CSA', 'Peruvian', 'Internet Cafes', 'Poutineries', 'Austria

In [20]:
print(len(final_category_set))

193


In [21]:
temp.show()

+--------------------+--------------------+
|         business_id|          categories|
+--------------------+--------------------+
|f9NumwFMBDn751xgF...|[Active Life, Gun...|
|Yzvjg0SayhoZgCljU...|[Health & Medical...|
|XNoUzKckATkOD1hP6...|[Pets, Pet Servic...|
|6OAZjbxqM5ol29BuH...|[Hardware Stores,...|
|51M2Kk903DFYI6gnB...|[Home Services, P...|
|cKyLV5oWZJ2NudWgq...|[Auto Repair, Aut...|
|oiAlXZPIFm2nBCt0D...|[Dry Cleaning & L...|
|ScYkbYNkDgCneBrD9...|[Auto Repair, Oil...|
|pQeaRpvuhoEqudo3u...|[Ethnic Food, Foo...|
|EosRKXIGeSWFYWwpk...|[Martial Arts, Gy...|
|MbZMmwo-eL0Jnm_Yb...|[Contractors, Lan...|
|7Dv4_HAxsxvadEsT5...|[Automotive, Auto...|
|M_guz7Dj7hX0evS67...|[Desserts, Food, ...|
|JjJs3o60uQCfctDjs...|[Contractors, Hom...|
|kOICO53wbOiOJcKuC...|[Beauty & Spas, T...|
|rqcOZePlVvJP9Etzl...|[Local Services, ...|
|uZuh51lXu7tsrC8RA...|[Shopping, Shoe S...|
|nIEhsGbw0vJuYl05b...|[Event Planning &...|
|edQoeeBFUTYGwnUSE...|[Auto Customizati...|
|Vwo64kNYDjKi98gUU...|[Weight Lo

In [22]:
print(temp.count())

208869


In [23]:
check = temp.rdd.map(lambda row : (row['business_id'], [x for x in row['categories'] if x in final_category_set])).toDF(["business_id", "categories"])
check = check.filter(f.size(check.categories) > 0)
check.show()
print(check.count())

+--------------------+--------------------+
|         business_id|          categories|
+--------------------+--------------------+
|pQeaRpvuhoEqudo3u...|[Food Trucks, Spe...|
|M_guz7Dj7hX0evS67...|[Desserts, Ice Cr...|
|CsLQLiRoafpJPJSkN...|[Grocery, Middle ...|
|mKTq1T_IAplDpHUcM...|[Food Delivery Se...|
|eBEfgOPG7pvFhb2wc...|[Cheesesteaks, Po...|
|lu7vtrp_bE9PnxWfA...|[Japanese, Fast F...|
|007Dg4ESDVacWcC4V...|[Organic Stores, ...|
|9sRGfSVEfLhN_km60...|[Persian/Iranian,...|
|pcaQDBM6r0PWTXfYZ...|           [Grocery]|
|DCsS3SgVFO56F6wRO...|[Donuts, Juice Ba...|
|vjTVxnsQEZ34XjYNS...|[Pretzels, Bakeri...|
|fnZrZlqW1Z8iWgTVD...|[Mexican, Fast Food]|
|rVBPQdeayMYht4Uv_...|           [Burgers]|
|98hyK2QEUeI8v2y0A...|  [Vietnamese, Soup]|
|fhNf_sg-XzZ3e7HEV...|[Burgers, America...|
|LoRef3ChgZKbxUio-...|    [Middle Eastern]|
|Ga2Bt7xfqoggTypWD...|           [Mexican]|
|_xOeoXfPUQTNlUAhX...|      [Coffee & Tea]|
|xFc50drSPxXkcLvX5...|[Burgers, Fast Fo...|
|tLpkSwdtqqoXwU0JA...|[Fast Food

In [24]:
df.show()

+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|           city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|     10913 Bailey Rd|[,,,,,,, True,, T...|f9NumwFMBDn751xgF...|Active Life, Gun/...|      Cornelius|[11:0-20:0, 10:0-...|      1|   35.4627242|    -80.8526119|The Range At Lake...|      28031|          36|  3.5|   NC|
|8880 E Via Linda,...|[,,,,,,,,,,, True...|Yzvjg0SayhoZgCljU...|Health & Medical,...|     Scottsdale|                null|  

In [25]:
restaurant_df = df.join(check, on=['business_id'], how='left_semi')

In [27]:
restaurant_df.show()

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|         business_id|             address|          attributes|          categories|         city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|--9e1ONYQuAa-CB_R...|3355 Las Vegas Bl...|[,, 'full_bar', {...|Seafood, Cajun/Cr...|    Las Vegas|[17:0-22:30, 17:0...|      1|    36.123183|     -115.16919|Delmonico Steakhouse|      89109|        1759|  4.0|   NV|
|-TGwtTiieh_JLml5_...|  10520 Yonge Street|[,,,,,,, True,,, ...|  Food, Coffee & Tea|Richmond Hill|[6:0-21:30, 6:0-2...|      0|    

In [38]:
t_df = restaurant_df.withColumn('price', restaurant_df.attributes.RestaurantsPriceRange2)
t_df = t_df[['business_id', 'name', 'address', 'city', 'state', 'categories', 'is_open', 'postal_code', 'stars', 'price', 'review_count']]
t_df.show()

+--------------------+--------------------+--------------------+-------------+-----+--------------------+-------+-----------+-----+-----+------------+
|         business_id|                name|             address|         city|state|          categories|is_open|postal_code|stars|price|review_count|
+--------------------+--------------------+--------------------+-------------+-----+--------------------+-------+-----------+-----+-----+------------+
|--9e1ONYQuAa-CB_R...|Delmonico Steakhouse|3355 Las Vegas Bl...|    Las Vegas|   NV|Seafood, Cajun/Cr...|      1|      89109|  4.0|    4|        1759|
|-TGwtTiieh_JLml5_...|           Starbucks|  10520 Yonge Street|Richmond Hill|   ON|  Food, Coffee & Tea|      0|    L4C 3C7|  3.5|    1|           8|
|-VAsjhmAbKF3Pb_-8...|          McDonald's|850, Rue Sainte-C...|     Montréal|   QC|Fast Food, Restau...|      1|    H2L 2E2|  2.0|    1|          12|
|-cxD1NimFldATDUsN...|      Coasta Cantina|4500 W Tropicana Ave|    Las Vegas|   NV|Mexican, R

In [40]:
# t_df.write.format('jdbc').mode('overwrite').option("url", "jdbc:mysql://database-1.cjy5mlvqnx2k.us-west-1.rds.amazonaws.com:3306/restaurant").option("dbtable", "restaurant_basic_info").option("user", "root").option("password", "wangyinuo696683").save()

In [41]:
check.show()

+--------------------+--------------------+
|         business_id|          categories|
+--------------------+--------------------+
|pQeaRpvuhoEqudo3u...|[Food Trucks, Spe...|
|M_guz7Dj7hX0evS67...|[Desserts, Ice Cr...|
|CsLQLiRoafpJPJSkN...|[Grocery, Middle ...|
|mKTq1T_IAplDpHUcM...|[Food Delivery Se...|
|eBEfgOPG7pvFhb2wc...|[Cheesesteaks, Po...|
|lu7vtrp_bE9PnxWfA...|[Japanese, Fast F...|
|007Dg4ESDVacWcC4V...|[Organic Stores, ...|
|9sRGfSVEfLhN_km60...|[Persian/Iranian,...|
|pcaQDBM6r0PWTXfYZ...|           [Grocery]|
|DCsS3SgVFO56F6wRO...|[Donuts, Juice Ba...|
|vjTVxnsQEZ34XjYNS...|[Pretzels, Bakeri...|
|fnZrZlqW1Z8iWgTVD...|[Mexican, Fast Food]|
|rVBPQdeayMYht4Uv_...|           [Burgers]|
|98hyK2QEUeI8v2y0A...|  [Vietnamese, Soup]|
|fhNf_sg-XzZ3e7HEV...|[Burgers, America...|
|LoRef3ChgZKbxUio-...|    [Middle Eastern]|
|Ga2Bt7xfqoggTypWD...|           [Mexican]|
|_xOeoXfPUQTNlUAhX...|      [Coffee & Tea]|
|xFc50drSPxXkcLvX5...|[Burgers, Fast Fo...|
|tLpkSwdtqqoXwU0JA...|[Fast Food

In [42]:
check.count()

78679

In [43]:
restaurant_category_list = list(final_category_set)
print(restaurant_category_list)

['Butcher', 'Dumplings', 'Fondue', 'Laotian', 'Cambodian', 'Pan Asian', 'Indian', 'German', 'Signature Cuisine', 'Basque', 'Polynesian', 'Shaved Snow', 'Juice Bars & Smoothies', 'Persian/Iranian', 'Canadian (New)', 'Sugar Shacks', 'Indonesian', 'Cheesesteaks', 'Donairs', 'Brazilian', 'Swiss Food', 'Catalan', 'Bakeries', 'Kombucha', 'Specialty Food', 'Taiwanese', 'Coffee & Tea', 'Soul Food', 'Eastern European', 'Hungarian', 'Buffets', 'Cideries', 'Delis', 'Gluten-Free', 'American (New)', 'Russian', 'Comfort Food', 'African', 'Hawaiian', 'International Grocery', 'Sri Lankan', 'Milkshake Bars', 'Syrian', 'Sushi Bars', 'Supper Clubs', 'Ice Cream & Frozen Yogurt', 'Czech/Slovakian', 'Backshop', 'Vegan', 'Food Court', 'Thai', 'Cafes', 'Food Delivery Services', 'Nicaraguan', 'Pretzels', 'Bubble Tea', 'Kebab', 'Modern European', 'Scottish', 'Tapas/Small Plates', 'Acai Bowls', 'Coffee & Tea Supplies', 'Fish & Chips', 'Middle Eastern', 'CSA', 'Peruvian', 'Internet Cafes', 'Poutineries', 'Austria

In [44]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, IntegerType

create_category_vector = check.rdd.map(lambda row : (row['business_id'], row['categories'], [1 if i in row['categories'] else 0 for i in restaurant_category_list])).toDF(['business_id', 'categories', 'binary_vector'])
create_category_vector.show()

+--------------------+--------------------+--------------------+
|         business_id|          categories|       binary_vector|
+--------------------+--------------------+--------------------+
|pQeaRpvuhoEqudo3u...|[Food Trucks, Spe...|[0, 0, 0, 0, 0, 0...|
|M_guz7Dj7hX0evS67...|[Desserts, Ice Cr...|[0, 0, 0, 0, 0, 0...|
|CsLQLiRoafpJPJSkN...|[Grocery, Middle ...|[0, 0, 0, 0, 0, 0...|
|mKTq1T_IAplDpHUcM...|[Food Delivery Se...|[0, 0, 0, 0, 0, 0...|
|eBEfgOPG7pvFhb2wc...|[Cheesesteaks, Po...|[0, 0, 0, 0, 0, 0...|
|lu7vtrp_bE9PnxWfA...|[Japanese, Fast F...|[0, 0, 0, 0, 0, 0...|
|007Dg4ESDVacWcC4V...|[Organic Stores, ...|[0, 0, 0, 0, 0, 0...|
|9sRGfSVEfLhN_km60...|[Persian/Iranian,...|[0, 0, 0, 0, 0, 0...|
|pcaQDBM6r0PWTXfYZ...|           [Grocery]|[0, 0, 0, 0, 0, 0...|
|DCsS3SgVFO56F6wRO...|[Donuts, Juice Ba...|[0, 0, 0, 0, 0, 0...|
|vjTVxnsQEZ34XjYNS...|[Pretzels, Bakeri...|[0, 0, 0, 0, 0, 0...|
|fnZrZlqW1Z8iWgTVD...|[Mexican, Fast Food]|[0, 0, 0, 0, 0, 0...|
|rVBPQdeayMYht4Uv_...|   

In [46]:
from pyspark.sql.functions import concat_ws
from pyspark.sql.types import StringType
db_vector = create_category_vector[['business_id', 'binary_vector']]

db_vector = db_vector.withColumn("binary_vector", concat_ws(",", "binary_vector"))
db_vector.show()

print(db_vector.select(['business_id']).dtypes)

+--------------------+--------------------+
|         business_id|       binary_vector|
+--------------------+--------------------+
|pQeaRpvuhoEqudo3u...|0,0,0,0,0,0,0,0,0...|
|M_guz7Dj7hX0evS67...|0,0,0,0,0,0,0,0,0...|
|CsLQLiRoafpJPJSkN...|0,0,0,0,0,0,0,0,0...|
|mKTq1T_IAplDpHUcM...|0,0,0,0,0,0,0,0,0...|
|eBEfgOPG7pvFhb2wc...|0,0,0,0,0,0,0,0,0...|
|lu7vtrp_bE9PnxWfA...|0,0,0,0,0,0,0,0,0...|
|007Dg4ESDVacWcC4V...|0,0,0,0,0,0,0,0,0...|
|9sRGfSVEfLhN_km60...|0,0,0,0,0,0,0,0,0...|
|pcaQDBM6r0PWTXfYZ...|0,0,0,0,0,0,0,0,0...|
|DCsS3SgVFO56F6wRO...|0,0,0,0,0,0,0,0,0...|
|vjTVxnsQEZ34XjYNS...|0,0,0,0,0,0,0,0,0...|
|fnZrZlqW1Z8iWgTVD...|0,0,0,0,0,0,0,0,0...|
|rVBPQdeayMYht4Uv_...|0,0,0,0,0,0,0,0,0...|
|98hyK2QEUeI8v2y0A...|0,0,0,0,0,0,0,0,0...|
|fhNf_sg-XzZ3e7HEV...|0,0,0,0,0,0,0,0,0...|
|LoRef3ChgZKbxUio-...|0,0,0,0,0,0,0,0,0...|
|Ga2Bt7xfqoggTypWD...|0,0,0,0,0,0,0,0,0...|
|_xOeoXfPUQTNlUAhX...|0,0,0,0,0,0,0,0,0...|
|xFc50drSPxXkcLvX5...|0,0,0,0,0,0,0,0,0...|
|tLpkSwdtqqoXwU0JA...|0,0,0,0,0,

In [47]:
# db_vector.write.format('jdbc').mode('overwrite').option("url", "jdbc:mysql://dsci551-project.cjy5mlvqnx2k.us-west-1.rds.amazonaws.com:3306/restaurants_info").option("dbtable", "restaurant_vector").option("user", "admin").option("password", "password").save()
db_vector.write.format('jdbc').mode('overwrite').option("url", "jdbc:mysql://database-1.cjy5mlvqnx2k.us-west-1.rds.amazonaws.com:3306/restaurant").option("dbtable", "restaurant_vector").option("user", "root").option("password", "wangyinuo696683").save()
restaurant_df.show()
restaurant_df.count()

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|         business_id|             address|          attributes|          categories|         city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|--9e1ONYQuAa-CB_R...|3355 Las Vegas Bl...|[,, 'full_bar', {...|Seafood, Cajun/Cr...|    Las Vegas|[17:0-22:30, 17:0...|      1|    36.123183|     -115.16919|Delmonico Steakhouse|      89109|        1759|  4.0|   NV|
|-TGwtTiieh_JLml5_...|  10520 Yonge Street|[,,,,,,, True,,, ...|  Food, Coffee & Tea|Richmond Hill|[6:0-21:30, 6:0-2...|      0|    

78679

In [29]:
# restaurant_df.write.format('jdbc').mode('overwrite').option("url", "jdbc:mysql://dsci551-project.cjy5mlvqnx2k.us-west-1.rds.amazonaws.com:3306/restaurants_info").option("dbtable", "restaurant_basic_info").option("user", "admin").option("password", "password").save()